데이터 로드

In [1]:
import multiprocessing 
from multiprocessing import Pool 
from functools import partial 
from data_loader_v2 import data_loader_v2
import os 
import pandas as pd
import numpy as np
import joblib

train_folder = 'train/'
train_label_path = 'train_label.csv'

In [ ]:
train_list = os.listdir(train_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [ ]:
def data_loader_all(func, files, folder='', train_label=None, event_time=15, nrows=75):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)  
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close() 
        pool.join() 
    combined_df = pd.concat(df_list)
    return combined_df

In [ ]:
train = data_loader_all(data_loader_v2, train_list, folder=train_folder, train_label=train_label, event_time=15, nrows=75)

In [ ]:
train=train.loc[:,train.std()!=0]

In [ ]:
X_train = train.drop(['label'], axis=1)
y_train = train['label']

In [ ]:
np.savez('train_ver3.npz', X=X_train, y=y_train)

load

In [4]:
train = np.load('train_fin.npz')

In [5]:
X = train['X']
y = train['y']

In [6]:
X.shape, y.shape

((49620, 3428), (49620,))

In [7]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import lightgbm as lgbm

In [14]:
#목적함수 생성
def lgbm_cv(learning_rate, num_leaves, max_depth, min_child_weight, colsample_bytree, feature_fraction, bagging_fraction):
    model = lgbm.LGBMClassifier(learning_rate=learning_rate,
                                num_leaves = int(round(num_leaves)),
                                max_depth = int(round(max_depth))),
                                min_child_weight = int(round(min_child_weight))),
                                colsample_bytree = colsample_bytree,
                                feature_fraction = max(min(feature_fraction, 1), 0),
                                bagging_fraction = max(min(bagging_fraction, 1), 0))
    RMSE = cross_val_score(model, X, y, scoring='accuracy', cv=5).mean()
    return -RMSE

In [15]:
# 입력값의 탐색 대상 구간
pbounds = {'learning_rate' : (0.00001, 0.3),
           'num_leaves': (50, 400),
           'max_depth': (-1, 9),
           'min_child_weight': (0, 100),
           'colsample_bytree': (0, 0.99),
           'feature_fraction': (0.0001, 0.99),
           'bagging_fraction': (0.0001, 0.99)
          }

In [16]:
#객체 생성
lgbmBO = BayesianOptimization(f = lgbm_cv,pbounds = pbounds, verbose = 2, random_state = 1 )

In [ ]:
# 반복적으로 베이지안 최적화 수행
# acq='ei'사용
# xi=0.01 로 exploration의 강도를 조금 높임
lgbmBO.maximize(init_points=4, n_iter = 16, acq='ei', xi=0.01)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | num_le... |
-------------------------------------------------------------------------------------


In [ ]:
lgbmBO.max # 찾은 파라미터 값 확인

In [ ]:
#목적함수 생성
def lgbm_cv(learning_rate, num_leaves, max_depth, min_child_weight, colsample_bytree, feature_fraction, bagging_fraction):
    model = lgbm.LGBMClassifier(learning_rate=learning_rate,
                                num_leaves = int(round(num_leaves)),
                                max_depth = int(round(max_depth))),
                                min_child_weight = int(round(min_child_weight))),
                                colsample_bytree = colsample_bytree,
                                feature_fraction = max(min(feature_fraction, 1), 0),
                                bagging_fraction = max(min(bagging_fraction, 1), 0))
    RMSE = cross_val_score(model, X, y, scoring='accuracy', cv=5).mean()
    return -RMSE

In [ ]:
#파라미터 적용
fit_lgbm = lgbm.LGBMClassifier(learning_rate=lgbmBO.max['params']['learning_rate'],
                               num_leaves = int(round(lgbmBO.max['params']['num_leaves'])),
                               max_depth = int(round(lgbmBO.max['params']['max_depth'])),
                               min_child_weight = int(round(lgbmBO.max['params']['min_child_weight'])),
                               colsample_bytree=lgbmBO.max['params']['colsample_bytree'],
                               feature_fraction = max(min(lgbmBO.max['params']['feature_fraction'], 1), 0),
                               bagging_fraction = max(min(lgbmBO.max['params']['bagging_fraction'], 1), 0),
                               )

In [ ]:
model = fit_lgbm.fit(X,y)

In [ ]:
joblib.dump(model_lgbm, 'model_lgbmBO.pkl')

In [ ]:
pred = model_lgbm.predict_proba(test)